# comparison between `ROSE` and `jitr`

We will use the high-fidelity solver in the [`ROSE` package](https://github.com/bandframework/rose). It is avilable on [pypi](https://pypi.org/project/nuclear-rose/) 

In [1]:
#!pip install nuclear-rose
import rose

In [2]:
from pathlib import Path

import corner

# import stuff for nice plotting
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
from numba import njit
from scipy import stats
from tqdm import tqdm

import jitr

We will use [exfor-tools](https://github.com/beykyle/exfor_tools) using an [x4i3](https://github.com/afedynitch/x4i3/) backend for grabbing data from [EXFOR](https://www-nds.iaea.org/exfor/) (requires 1GB of disk space to download).

### Let's find some data to compare to
Let's grab some data from [EXFOR](https://www-nds.iaea.org/exfor/). We will look at $d\sigma_{el}/d\Omega$ for $(n,n)$ and $(p,p)$.

In [3]:
# target to consider
Pb208 = (208, 82)
proton = (1, 1)
neutron = (1, 0)

# for plotting differential xs
angles = np.linspace(0.1, np.pi, 100)

In [4]:
sys = jitr.reactions.ProjectileTargetSystem(
    channel_radius=10 * np.pi,
    lmax=15,
    mass_target=jitr.utils.kinematics.mass(*Pb208),
    mass_projectile=jitr.utils.kinematics.mass(*neutron),
    Ztarget=Pb208[1],
    Zproj=0,
    coupling=jitr.reactions.system.spin_half_orbit_coupling,
)

In [5]:
Ecm = 12
mu, Elab, k, eta = jitr.utils.kinematics.classical_kinematics_cm(
    sys.mass_target, sys.mass_projectile, Ecm, sys.Zproj * sys.Ztarget
)

In [6]:
ws = jitr.xs.ElasticXSWorkspace(
    projectile=neutron,
    target=Pb208,
    sys=sys,
    Ecm=Ecm,
    k=k,
    mu=mu,
    eta=eta,
    local_interaction_scalar=jitr.reactions.KD_scalar,
    local_interaction_spin_orbit=jitr.reactions.KD_spin_orbit,
    solver=jitr.rmatrix.Solver(50),
    angles=angles,
)

In [7]:
kduq = rose.InteractionSpace(
    coordinate_space_potential=rose.koning_delaroche.KD_simple,
    n_theta=rose.koning_delaroche.NUM_PARAMS,
    is_complex=True,
    spin_orbit_term=rose.koning_delaroche.KD_simple_so,
    energy=Ecm,
    mu=mu,
    k=k,
    l_max=15,
)
kduq_rose_solver = rose.ScatteringAmplitudeEmulator.HIFI_solver(
    interaction_space=kduq,
    angles=angles,
)

In [8]:
# we have 416 samples from the KDUQ posterior
num_samples_kduq = 415
param_files = [
    Path(f"./../../src/data/KDUQFederal/{i}/parameters.json")
    for i in range(0, num_samples_kduq)
]

# load each one
rose_omps = [
    rose.koning_delaroche.KDGlobal(rose.Projectile.neutron, Path(param_file))
    for param_file in param_files
]
# load each one
jitr_omps = [
    jitr.reactions.kduq.KDGlobal((1, 0), Path(param_file)) for param_file in param_files
]

In [9]:
coulomb_params, scalar_params, spin_orbit_params = jitr_omps[7].get_params(*Pb208, mu, Elab, k)

In [10]:
R_c, params = rose_omps[7].get_params(*Pb208, mu, Elab, k)

In [11]:
params[-len(spin_orbit_params):] - spin_orbit_params

array([0., 0., 0., 0., 0., 0.])

In [12]:
params[:len(scalar_params)] - scalar_params

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
R_c - coulomb_params[1]

0

In [14]:
spr, smr = kduq_rose_solver.exact_smatrix_elements(params)

In [15]:
spj, smj = ws.xs(   
    args_scalar=scalar_params,
    args_spin_orbit=spin_orbit_params
)

In [ ]:
smr[1:] - smj

In [21]:
spr - spj[:14]

array([-4.14080095e-04-1.41287480e-04j,  1.13315184e-03-2.27479680e-04j,
       -1.30128671e-03+9.71931937e-04j,  5.69529162e-04-9.92334772e-04j,
       -4.26338669e-04-2.99288798e-04j,  2.27661873e-03-2.62388369e-04j,
        5.71175877e-04+1.71185683e-03j,  3.16205361e-04-8.22301764e-05j,
        2.88522691e-04+1.29959074e-04j, -1.15358891e-05-1.27590187e-05j,
       -5.39327323e-08-4.15380830e-08j,  6.01526373e-09+3.32076923e-08j,
        3.11316861e-10+1.54303406e-08j,  1.02066133e-11+9.46187445e-09j])

In [16]:
dsdo_jitr = ws.xs(
    args_scalar=scalar_params,
    args_spin_orbit=spin_orbit_params
)

In [17]:
dsdo_rose = kduq_rose_solver.exact_xs(params)

In [18]:
dsdo_rose.dsdo - dsdo_jitr.dsdo

AttributeError: 'tuple' object has no attribute 'dsdo'